In [4]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/marcm\git-files\Ironhack Data Analytics.json"
from google.cloud import bigquery

In [5]:
bigquery.Client()

In [6]:
client = bigquery.Client()

## Challenge 1 - Most Profiting Authors

In [5]:
#Step 1: Calculate the royalties of each sales for each author

query = '''
SELECT
    ta.title_id,
    ta.au_id,
    t.price * s.qty * (t.royalty/100) * (ta.royaltyper/100) AS Sales_royalty
    
FROM
    `ironhack-data-analytics.publications.titleauthor`ta
    
JOIN 
    `ironhack-data-analytics.publications.titles`t
ON 
    ta.title_id = t.title_id
JOIN
    `ironhack-data-analytics.publications.sales`s
ON
    t.title_id = s.title_id

GROUP BY
    1,2,3
    
ORDER BY
    3 DESC
    
    
'''

query_job = client.query(query=query)
df=query_job.to_dataframe()
df

,title_id,au_id,Sales_royalty
0,PC1035,238-95-7766,110.160
1,TC3218,807-91-6654,83.800
2,PC8888,427-17-2319,50.000
3,PC8888,846-92-7186,50.000
4,PS2091,998-72-3567,49.275
5,PS2091,899-46-2035,49.275
6,TC4203,648-92-1872,33.460
7,PS1372,756-30-7391,32.385
8,PS3333,172-32-1176,29.985
9,BU7832,274-80-9391,29.985


In [6]:
#Step 2: Aggregate the total royalties for each title for each author 

query = '''
SELECT
    ta.title_id,
    ta.au_id,
    SUM(t.price * s.qty * (t.royalty/100) * (ta.royaltyper/100)) AS Aggregated_royalties
    
FROM
    `ironhack-data-analytics.publications.titleauthor`ta
    
JOIN 
    `ironhack-data-analytics.publications.titles`t
ON 
    ta.title_id = t.title_id
JOIN
    `ironhack-data-analytics.publications.sales`s
ON
    t.title_id = s.title_id

GROUP BY
    1,2
    
ORDER BY
    3 DESC
    
    
'''    
            

query_job = client.query(query=query)
df=query_job.to_dataframe()
df

,title_id,au_id,Aggregated_royalties
0,PC1035,238-95-7766,110.160
1,TC3218,807-91-6654,83.800
2,PS2091,899-46-2035,70.956
3,PS2091,998-72-3567,70.956
4,PC8888,846-92-7186,50.000
5,PC8888,427-17-2319,50.000
6,TC4203,648-92-1872,33.460
7,PS1372,756-30-7391,32.385
8,PS3333,172-32-1176,29.985
9,BU7832,274-80-9391,29.985


In [9]:
#Step 2: Aggregate the total royalties for each title for each author using SUBQUERY

query2= '''

SELECT  
    ta.title_id,
    ta.au_id,
   (SELECT SUM((t.price * s.qty * (t.royalty/100) * (ta.royaltyper/100)))  
        FROM 
            `ironhack-data-analytics.publications.titles`t
        JOIN
            `ironhack-data-analytics.publications.sales`s
        ON
            t.title_id = s.title_id) AS Aggregated_royalties
         
FROM
    `ironhack-data-analytics.publications.titleauthor`ta
        
ORDER BY
    3 DESC
    
'''
query_job = client.query(query=query2)
df=query_job.to_dataframe()
df


,title_id,au_id,Aggregated_royalties
0,BU2075,213-46-8915,777.60500
1,BU7832,274-80-9391,777.60500
2,PS3333,172-32-1176,777.60500
3,MC2222,712-45-1867,777.60500
4,TC4203,648-92-1872,777.60500
5,PS2106,998-72-3567,777.60500
6,PC9999,486-29-1786,777.60500
7,TC3218,807-91-6654,777.60500
8,PC1035,238-95-7766,777.60500
9,PS7777,486-29-1786,777.60500


In [10]:
# Step 3: Calculate the total profits of each author

query = '''

SELECT
    ta.au_id,
    (SELECT SUM((t.price * s.qty * (t.royalty/100) * (ta.royaltyper/100)) + t.advance)
        
        FROM 
            `ironhack-data-analytics.publications.titles`t
        JOIN
            `ironhack-data-analytics.publications.sales`s
        ON
            t.title_id = s.title_id) AS Profits
FROM
    `ironhack-data-analytics.publications.titleauthor`ta
        
ORDER BY
    2 DESC
    
LIMIT 
    3
    
'''
query_job = client.query(query=query)
df=query_job.to_dataframe()
df
    
    
 

,au_id,Profits
0,274-80-9391,123002.605
1,213-46-8915,123002.605
2,172-32-1176,123002.605


## Challenge 2

In [26]:
query = '''

CREATE TABLE publications.most_profiting_authors_m(
    au_id STRING NOT NULL,
    Profits NUMERIC NOT NULL)
       
'''
query_job = client.query(query=query)
df=query_job.to_dataframe()
df



Conflict: 409 GET https://www.googleapis.com/bigquery/v2/projects/ironhack-data-analytics/queries/6960a1d1-8cf8-4982-a248-cd411d9ba6dd?maxResults=0&location=US: Already Exists: Table ironhack-data-analytics:publications.most_profiting_authors_m

In [18]:
query_job.result()

In [24]:
query_insert = '''
INSERT INTO 
    publications.most_profiting_authors_m(au_ID, Profits)
SELECT 
    ta.au_id,
    (SELECT SUM((t.price * s.qty * (t.royalty/100) * (ta.royaltyper/100)) + t.advance)
        
        FROM 
            `ironhack-data-analytics.publications.titles`t
        JOIN
            `ironhack-data-analytics.publications.sales`s
        ON
            t.title_id = s.title_id) AS Profits
FROM
    `ironhack-data-analytics.publications.titleauthor`ta
        
ORDER BY
    2 DESC
'''
query_job = client.query(query=query_insert)
df=query_job.to_dataframe()
df

BadRequest: 400 Query column 2 has type FLOAT64 which cannot be inserted into column Profits, which has type NUMERIC at [4:1]

In [ ]:
#He creado una tabla a cuya columna Profits he asignado una variable de tipo NUMERIC
#pero al intentar añadir los valores, al ser tipo FLOAT no me los acepta y
#no consigo modificar la tabla para que los acepte.